In [1]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.8 MB/s eta 0:00:00


In [2]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from datasets import Dataset
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments

In [ ]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 89.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.3 MB/s eta 0:00:00


In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer, AdamW
from tqdm import tqdm
import pandas as pd

data = pd.read_csv('train.csv', skip_blank_lines=True)

# Define your custom dataset class
class MyDataset(Dataset):
    def __init__(self, texts, targets, tokenizer, max_length=512):
        self.texts = texts
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        target = self.targets[idx]

        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        labels = self.tokenizer(
            target,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )["input_ids"]

        return {"input_ids": encoding["input_ids"].squeeze(), "attention_mask": encoding["attention_mask"].squeeze(), "labels": labels.squeeze()}

# Load your dataset
# Replace the following lines with loading your own dataset
texts = list(data['Question'])[:200]
targets = list(data['Equation'])[:200]

# Tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Create a DataLoader for your dataset
dataset = MyDataset(texts, targets, tokenizer)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# Optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=0.001)
num_epochs = 4

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

for epoch in range(num_epochs):
    total_loss = 0
    total_correct = 0
    total_samples = 0
    for batch in tqdm(dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

        model.eval()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        predicted_ids = torch.argmax(outputs.logits, dim=-1)
        correct = (predicted_ids == labels).sum().item()
        total_correct += correct
        total_samples += labels.numel()

        loss = outputs.loss
        total_loss+=loss.item()
        loss.backward()
        optimizer.step()


    print(f"Loss {total_loss/len(dataloader)} Accuracy : {total_correct/total_samples}")

# Save the fine-tuned model
model.save_pretrained("your_fine_tuned_model")
tokenizer.save_pretrained("your_fine_tuned_model")


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

...

Epoch 3/4: 100%|██████████| 100/100 [19:11<00:00, 11.51s/it]


Loss 0.0072623097925679754 Accuracy : 0.997509765625


Epoch 4/4: 100%|██████████| 100/100 [18:53<00:00, 11.34s/it]


Loss 0.006397342188283801 Accuracy : 0.997734375


('your_fine_tuned_model/tokenizer_config.json',
 'your_fine_tuned_model/special_tokens_map.json',
 'your_fine_tuned_model/spiece.model',
 'your_fine_tuned_model/added_tokens.json')

In [14]:
def calculate_accuracy(data,model):
    total_correct = 0
    for i,input_text in enumerate(data['Question']):
        input_ids = tokenizer(input_text, return_tensors="pt").input_ids
        model.eval()
        output_ids = model.generate(input_ids)
        output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        # print(f"Predicted : {output_text}")
        # print(f"True : {data['Equation'][i]}")
        if output_text == data['Equation'][i]:
            total_correct+=1
    return total_correct/len(data)

Test evaluation

In [10]:
test_data = pd.read_csv('dev.csv')
texts = list(test_data['Question'])[:100]
targets = list(test_data['Equation'])[:100]
dataset_test = MyDataset(texts, targets, tokenizer)
dataloader_test = DataLoader(dataset_test, batch_size=1, shuffle=True)
calculate_accuracy(test_data, model)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Predicted : - number0 number1
True : - number0 number1
Predicted : - number0 number1
True : - number1 number0
Predicted : / + number0 number1
True : - number1 number0
Predicted : - number0 number1
True : - number1 number0
Predicted : - + number0 number1 number1
True : - number0 number1
Predicted : - + number0 number1 number1
True : + number0 number1
Predicted : + + number0 number1 number2
True : + + number0 number1 number2
Predicted : + + number0 number1 number2
True : + number0 number1
Predicted : + + number0 number1 number2
True : + number0 number2
Predicted : + + number0 number1 number2
True : + number1 number2
Predicted : + + number0 number1 number2
True : - number0 number1
Predicted : + + number0 number1 number2
True : - number0 number2
Predicted : + + number0 number1 number2
True : - number1 number2
Predicted : + + number0 number1 number2
True : - + number0 number1 number2
Predicted : + + number0 number1 number2
True : + + number0 number1 number2
Predicted : + + number0 number1 n

0.064

In [15]:
calculate_accuracy(data, model)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


0.51